In [1]:
import rubrix as rb
import pandas as pd

# Create a basic text classification record
#record = rb.TextClassificationRecord(text="Hello world, this is me!")

# Upload (log) the record to the Rubrix web app
#rb.log(record, "my_first_record")

In [3]:
mydf = pd.read_csv('../OffensiveTweets1.csv')

In [4]:
sum(mydf.duplicated())

0

In [5]:
mydf = mydf.rename(columns={"tweet": "text"})

In [9]:
# Read DataFrame into a Rubrix Dataset
offensiveData = rb.read_pandas(mydf, task="TextClassification")

In [11]:
rb.log(offensiveData, "project-weaklabel")

  0%|          | 0/45033 [00:00<?, ?it/s]

45033 records logged to http://localhost:6900/datasets/rubrix/project-weaklabel


BulkResponse(dataset='project-weaklabel', processed=45033, failed=0)

In [17]:
from rubrix.labeling.text_classification.weak_labels import  WeakLabels
weak_labels = WeakLabels(dataset="project-weaklabel")

Preparing rules:   0%|          | 0/11 [00:00<?, ?it/s]

Applying rules:   0%|          | 0/45033 [00:00<?, ?it/s]

In [37]:

groupoff = weak_labels.show_records(labels=['Offensive-Group'])[['text']]
groupoff['weaklabel'] = 'Offensive-Group'

indoff = weak_labels.show_records(labels=['Offensive-Ind'])[['text']]
indoff['weaklabel'] = 'Offensive-Ind'

untaroff = weak_labels.show_records(labels=['Offensive-Untargetted'])[['text']]
untaroff['weaklabel'] = 'Offensive-Untargetted'

unoff = weak_labels.show_records(labels=['Not-Offensive'])[['text']]
unoff['weaklabel'] = 'Not-Offensive'

weakdf = pd.concat([groupoff, indoff, untaroff, unoff], ignore_index=True)
weakdf.head()

,text,weaklabel
0,Revolution Is Homegrown. Always. എന്ന് വാരിയ...,Offensive-Group
1,വെള്ളപ്പൊക്കത്തിന് ഇടയ്ക്കു നോർത്തികളെ കാണിക്ക...,Offensive-Group
2,Babri is a കുണ്ടൻ,Offensive-Group
3,ഒരു കുണ്ടൻ ലുക്ക്,Offensive-Group
4,വല്ലാത്ത നേരകം ആണ് ഉസ്താതെ... ഒരു ചോദ്യം കുണ്...,Offensive-Group


In [48]:
weakdf['text'] = weakdf['text'].drop_duplicates(keep = 'first' )

In [49]:
weakdf.weaklabel.value_counts()

Offensive-Ind            8185
Offensive-Group          1004
Offensive-Untargetted      53
Not-Offensive              37
Name: weaklabel, dtype: int64

In [52]:
from sklearn.utils import shuffle

weakdf = shuffle(weakdf, random_state= 44)

In [53]:
weakdf.head()

,text,weaklabel
1642,നീ ഊമ്പി ഉണ്ടാക്കിയതാണോ.. മിടുക്കനാണല്ലോ,Offensive-Ind
4334,ചുരുക്കിപ്പറഞ്ഞാൽ നരേന്ദ്ര മോഡിയോട് 'നിന്റെ തന...,Offensive-Ind
14,ഇത് പാണക്കാട് ഷിഹാബു തങ്ങളുടെ മോന്റെ കുണ്ടൻ....,Offensive-Group
6972,നീ ഏതപ്പീ ? പൂണൂല്‍ കണ്ടിട്ടു മനസിലായില്ലെട ജ...,Offensive-Ind
8500,"നിന്റെ വകയിൽ ഉള്ള തന്ത , o...",Offensive-Ind


In [ ]:
weakdf.to_csv('WeakLabeled.csv', index =False)

In [3]:

df = pd.read_csv('../WeakLabelled/WeakLabeled.csv')

In [4]:
df.head()


,text,weaklabel
0,"""കൊക്കിലൊതുങ്ങുന്നതേ കൊത്താവൂ"" ഒക്കെ ഏതെങ്കിലു...",Offensive-Ind
1,ഞാനും മൈരൻ മമ്മദും അതിന് നിന്റെ വെടിച്ചി തള്...,Offensive-Ind
2,ആൻഡ് ഒട്ടമെന്റി ഈസ് എ മൈരൻ,Offensive-Ind
3,എത്ര നാളായി ഈ അവരാധി എന്നെ ബ്ലോക്ക് ചെയ്തിട്ട്...,Offensive-Ind
4,"ഒന്നും പറയണ്ട,,,മുഴു നടന്റെ പടത്തെ അഭിപ്രായം ...",Offensive-Ind


In [5]:
df['weaklabel'].value_counts()

Offensive-Ind            7353
Offensive-Group          1514
Offensive-Untargetted     500
Not-Offensive              37
Name: weaklabel, dtype: int64

In [35]:
import re
def engcheck(x):
    check =  re.compile(r'.*[a-zA-Z]+.*')
    if check.match(x):
        return "english"
    else:
        return "malayalam"

In [36]:
mydf['isalpha'] = mydf['text'].map(engcheck)

In [38]:
mydf.isalpha.value_counts()

malayalam    37491
english       9141
Name: isalpha, dtype: int64

In [41]:
pd.options.display.max_colwidth = 675
mydf[mydf['isalpha']=='english']

,text,isalpha
3,"പിഴച്ചു പെറ്റ മറിയത്തിന്റെ കഥ പറഞ്ഞുകൊടുത്ത അമ്മയോട് ജോൺ ചോദിച്ചു ആരാ അമ്മേ എന്റെ തന്ത, അമ്മ പറഞ്ഞു Still looking for it. അതെ അമ്മയുടെ വാക്കും ഉരുവിട്ട്കൊണ്ട് അവൻ ഇപ്പോഴും തിരയുകയാണ് 🤷‍♂️",english
5,ഞാ കാര് തി എന്നെ കൽ യാനo കഴി ക് ന് ആയി ര് കും എന്ന് ഞാ കൊച്ച് അല്ലേ ഞാ പേടി ച് പോയി ട്ട് എങ്ങണ്ട് ഞാ ബ്ലോക്ക് ചെയതെ ano പക്ഷേ ആ അളിയാ നെ ഞാ tl പോലു കണ്ട് ട്ട് പോലു ഇല്ല്യാ എന്നിട്ട് ആ നാറി റൗഡി കുഞ്ഞ് നെ പറ്റി ട്വീറ്റ് ഇട്ടേ ഞാ comt ഇട്ട് ട്ട് ഒര് day കഴി ഞ്ട്ട് reply തന്ന് elly,english
27,"Kurach thendis njagalde veednte frontiludE poonnind, innik avre all Kann edutha kandauunda athream virthi കേട്ട തെണ്ടികൾ annu, ഇന്ന് ippo ദേ, njan purath irrikumbo നാണം ഇല്ലാതെ vayanoki പോൺ, ചെറ്റ പന്നിസ്",english
42,A real threat to humanity. A polayadi monodu enthu paranjittum karayam illa. Let karma save him.,english
49,നിന്റെ തന്ത തള്ളയെ party ആപ്പീസിലെ ചകക്കാൾ കേറി പണിയുമ്പോൾ നിലത്തു കിടന്ന് മോങ്ങുന്ന അതെ ഭാവം 🤭🤭🤭,english
...,...,...
46612,"തൊട്ടാവാടി പെണ്ണാണെന്ന് കരുതി കാട്ടുകോഴി ടിയെമ്മില്‍ തൊട്ടാല്‍ വാടുമോ ഡാര്‍ലിംഗ്?? :P തൊട്ടു:ഇല്ലെടാ തെണ്ടി,തൊട്ടാലെണീക്കും.. :D",english
46617,"അച്ഛമ്മ - ടാ, ഇവർ ഈ പറയുന്നതൊക്കെ നേരാണോ? കുഞ്ഞുട്ടൻ - ഏഹ് ?എന്താ ? ആരാ ഇവരൊക്കെ? (യാതൊരു പരിചയവും ഇല്ലാത്ത ഭാവത്തിൽ) ഷണ്മുഖൻ - ഞാൻ നിന്റെ തന്ത, ഇവരൊക്കെ നിന്റെ മറ്റ് തന്തമാർ. #മേഘം ഹനീഫക്ക പോയിട്ട് 9 വർഷങ്ങൾ ആയെന്നു വിശ്വസിക്കാൻ പ്രയാസം. Most lovable actor of our times. 💕",english
46619,"UDF നെതിരെ ആയിരുന്നു ഞങ്ങളുടെ ബാർ സമരം എന്ന് ബിജയാ രാഗവാടാൻ സർ...😂😂 കേരളീയരായ നാം കഴിക്കുന്നത് മനുഷ്യ ഭക്ഷണം ആണോ, കഴുത പുല്ല് ആയിരിന്നില്ലേ എന്നാണ്.😏 തന്ത ആരെന്നു ചോദിച്ചാൽ കൂടെ കാണുന്നവനെ ചൂണ്ടി കാണിക്കുന്നവനെ എന്ത് ചെയ്യണം എന്നു കൊച്ചു പാലാ കാണിച്ചു കൊടുത്തിട്ടുണ്ട്. 😂😂",english
46624,ഇറങ്ങി പോടാ തെണ്ടി ജാഥവെ #RemoveJadhav,english


In [28]:
a = "Edi malare നായിൻ്റെ mole chetta നാറി ente art thott കളിച്ചാൽ undalodi naari panna errape ninte മുന്നിലെ രണ്ട് pall kannuledi thendi myre"

check =  re.compile(r'[a-zA-Z]*')

print(check.match(a))

<re.Match object; span=(0, 3), match='Edi'>
